In [ ]:
from unsloth import FastLanguageModel
import numpy as np
import random
from peft import LoftQConfig
import torch
from tool import dataset_DEAL
seed = 3407
WORK = 13      #  3:仇恨目标搜索微调  31:仇恨目标奖励微调   12 俚语分析微调
WORKFILE = './DATA/train.json'  # 训练数据集
MODEL = "./OUT/DQ7_S133"  # 模型路径
OUTDIR = 'OUT/DQ7_2'  # 输出路径
max_seq_length = 2048 
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# !python rrrrrrrrrrrrr.py --work 12 --workfile ./DATA/train.json --model ./OUT/DQ7_S133 --outdir OUT/DQ7_2 
# !python rrrrrrrrrrrrr.py --work 12 --workfile ./outputnew_3_CC.json --model ./OUT/DQ7_2 --outdir OUT/DQ7_3 
# !python rrrrrrrrrrrrr.py --work 12 --workfile ./outputnew_4_CC.json --model ./OUT/DQ7_3 --outdir OUT/DQ7_4 
# !python rrrrrrrrrrrrr.py --work 3 --workfile ./DATA/test.json --model ./OUT/DQ7_4 --outdir OUT/DQ7_5 

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL,
    max_seq_length = max_seq_length, # Choose any! We auto support RoPE Scaling internally!
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
)
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    lora_alpha = 256, # Choose any number > 0 ! Suggested 16, 32, 64, 128
    lora_dropout = 0.3, # Choose any number between 0 and 1 ! Suggested 0.05, 0.1
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj",],
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = seed,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = LoftQConfig(loftq_bits = 4, loftq_iter = 16)
    # loftq_config = None
)
dataset_train,dataset_test = dataset_DEAL(WORKFILE,WORK,seed)
def generate_r1_prompt(prompt):
    # return {"prompt": tokenizer.apply_chat_template(prompt, tokenize=False, continue_final_message=True)+'<|im_end|>', "target": target}
    return {"prompt": tokenizer.apply_chat_template(prompt, tokenize=False, continue_final_message=True)+'<|im_end|>'}
dataset_train = dataset_train.map(lambda x: generate_r1_prompt(x["prompt"]))

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import os
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, 
        max_steps = -1, # 移除或设置为-1，让模型训练完整一个epoch
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = seed,
        output_dir = "OUT",
    ),
)
trainer_stats = trainer.train()
os.makedirs("./OUT", exist_ok=True)
model = model.merge_and_unload()
model.save_pretrained(OUTDIR)  
tokenizer.save_pretrained(OUTDIR)

In [ ]:
# from tool import PRO
# # _,dataset_test = dataset_DEAL(WORKFILE,WORK*10 + 1,seed)
# # def generate_r1_prompt(prompt, target):
#     # return {"prompt": tokenizer.apply_chat_template(prompt, tokenize=False, continue_final_message=True), "target": target}
# # dataset_test = dataset_test.map(lambda x: generate_r1_prompt(x["prompt"], x["target"]))
# FastLanguageModel.for_inference(model)
# text = """<|im_start|>system
# 进行'俚语分析'任务。从'社交媒体发言'中分析作者使用的俚语<|im_end|>
# <|im_start|>user
# ‘默’指黑狗，指黑人
# 社交媒体发言:郭楠 | 也配碰瓷你默大爷啊😓<|im_end|>
# <|im_start|>assistant
# """
# inputs = tokenizer([text], return_tensors = "pt").to("cuda")

# # outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# # tokenizer.batch_decode(outputs)
# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 300)

In [ ]:
# # model.save_pretrained("lora_model")  # Local saving
# # tokenizer.save_pretrained("lora_model")
# # model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# # tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

# if True:
#     from unsloth import FastLanguageModel
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
#         max_seq_length = max_seq_length,
#         dtype = None,
#         load_in_4bit = True,
#     )
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# # alpaca_prompt = You MUST copy from above!

# inputs = tokenizer(
# [
#     prompt.format(
#         "What is a famous tall tower in Paris?", # instruction
#         "", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
# # unsloth/Qwen2.5-3B-Instruct-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-unsloth-bnb-4bit
# # unsloth/Qwen2.5-3B-Instruct-unsloth-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-1M-bnb-4bit
# # unsloth/Qwen2.5-7B-Instruct-1M-unsloth-bnb-4bit
# # @title Show current memory stats
# gpu_stats = torch.cuda.get_device_properties(0)
# start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
# print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
# print(f"{start_gpu_memory} GB of memory reserved.")
# # @title Show final memory and time stats
# used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
# used_percentage = round(used_memory / max_memory * 100, 3)
# lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
# # print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
# # print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
# print(f"Peak reserved memory = {used_memory} GB.")
# print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
# print(f"Peak reserved memory % of max memory = {used_percentage} %.")
# print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")